## Análise Espacial - MBA em DSA - USP-ESALQ - Prof. Dr. Rafael de Freitas Souza

## Aula 3 - Vizinhanças e Esda - Exploratory Spatial Data Analysis

## Importando as bibliotecas. Não esqueça de verificar quais vc precisa instalar.

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.formula.api as sm

# para gráficos
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import esda

In [ ]:
import geopandas as gpd
#import pysal as ps
#import splot
#import mapclassify as mc
#from libpysal import weights
#from esda import Moran, Moran_local, G_local
#from splot.esda import plot_moran, moran_scatterplot, lisa_cluster, plot_local_autocorrelation 

# Estabelecendo vizinhanças

In [ ]:
#Carregando o shapflie do estado de SP
shp_sp = gpd.read_file('shapefiles/estado_sp.shp', encoding = 'windows-1252')

In [ ]:
#Plotando o shapefile
shp_sp.plot(figsize = (5,5))

# Estabelecendo vizinhanças por contiguidade, critério QUEEN:

In [ ]:
# nomes das variáveis
shp_sp.columns

## Para vizualizar as vizinhanças, o centroide deve ser definido em metros e não em graus. Portanto, deve-se alterar a geometria (o CRS) dos dados

In [ ]:
shp_sp_m = shp_sp.to_crs(5880)

In [ ]:
import libpysal
from libpysal.weights import Queen, Rook, KNN, Kernel, weights

In [ ]:
w_queen = Queen.from_dataframe(shp_sp_m)

### Veja que é identificada a presença de uma ilha, de id = 246. Significa que este município não tem vizinhança com nenhum outro

In [ ]:
# Para saber qual é o município, é só acessar o index do shapefile de SP
shp_sp['NM_MUNICIP'][[246]]

# Visualizando a vizinhança estabelecida

In [ ]:
ax = shp_sp_m.plot(edgecolor='grey', facecolor='w', figsize = (25,25))
f,ax = w_queen.plot(shp_sp_m, ax=ax,
        edge_kws=dict(color='r', linestyle=':', linewidth=1),
        node_kws=dict(marker='o'))
ax.set_axis_off()

## Informações relevantes sobre a vizinhança queen estabelecida:

In [ ]:
#Alguns atributos da matrix
print("Número de regiões: ", w_queen.n) 
print("Número de conexões:", w_queen.nonzero)
print("Porcentagem de conexões:", w_queen.pct_nonzero)
print("Município ilha: ", w_queen.islands) # Municipios que são ilhas (sem conector)
print("Maior número de conexões: ", w_queen.max_neighbors) 
print("Média número de conexões: ", w_queen.mean_neighbors) 
print("Menor número de conexões: ", w_queen.min_neighbors) 

In [ ]:
# Qual é a ilha?
shp_sp_m['NM_MUNICIP'][w_queen.islands]

In [ ]:
# Dicionário com o número de conexões de cada município
w_queen.cardinalities

In [ ]:
#Distribuição por número de conexões
w_queen.histogram

In [ ]:
w_queen.neighbors

### Vizualização da matriz Queen

In [ ]:
pd.DataFrame(*w_queen.full()).astype(int)

# Estabelecendo vizinhanças por contiguidade, critério ROOK:

In [ ]:
w_rook = Rook.from_dataframe(shp_sp_m)

## Visualizando a vizinhança estabelecida

In [ ]:
ax = shp_sp_m.plot(edgecolor='grey', facecolor='w', figsize = (25,25))
f,ax = w_rook.plot(shp_sp_m, ax=ax,
        edge_kws=dict(color='r', linestyle=':', linewidth=1),
        node_kws=dict(marker='o'))
ax.set_axis_off()

## Informações relevantes sobre a vizinhança queen estabelecida:

In [ ]:
#Alguns atributos da matrix
print("Número de regiões: ", w_rook.n) 
print("Número de conexões:", w_rook.nonzero)
print("Porcentagem de conexões:", w_rook.pct_nonzero)
print("Município ilha: ", w_rook.islands) # Municipios que são ilhas (sem conector)
print("Maior número de conexões: ", w_rook.max_neighbors) 
print("Média número de conexões: ", w_rook.mean_neighbors) 
print("Menor número de conexões: ", w_rook.min_neighbors) 

In [ ]:
#Distribuição por número de conexões
w_rook.histogram

# Vizinhanças por Distância Geográfica

In [ ]:
# Carregando o shapefile
shp_ba = gpd.read_file('shapefiles/ba_state.shp', encoding = 'windows-1252')

In [ ]:
shp_ba.head()

In [ ]:
vizinhos_distancia = Kernel.from_dataframe(shp_sp_m)

In [ ]:
vizinhos_distancia.function

In [ ]:
vizinhos_distancia.bandwidth[0:5]

In [ ]:
# Build weights with adaptive bandwidth
w_adaptive = Kernel.from_dataframe(
    shp_sp_m,fixed=False, k=1
)
# Print first five bandwidth values
w_adaptive.bandwidth[:5]

In [ ]:
# Create full matrix version of weights
full_matrix, ids = w_adaptive.full()
# Set up figure with two subplots in a row
f,ax = plt.subplots(
    1, 2, figsize=(12,6), subplot_kw=dict(aspect='equal')
)
# Append weights for first polygon and plot on first subplot
shp_sp_m.assign(
    weight_0 = full_matrix[0]
).plot("weight_0", cmap='plasma', ax=ax[0])
# Append weights for 18th polygon and plot on first subplot
shp_sp_m.assign(
    weight_18 = full_matrix[17]
).plot("weight_18", cmap='plasma', ax=ax[1])
# Add centroid of focal tracts
shp_sp_m.iloc[[0], :].centroid.plot(
    ax=ax[0], marker="*", color="k", label='Focal Tract'
)
shp_sp_m.iloc[[17], :].centroid.plot(
    ax=ax[1], marker="*", color="k", label='Focal Tract'
)
# Add titles
ax[0].set_title("Kernel centered on first tract")
ax[1].set_title("Kernel centered on 18th tract")
# Remove axis
[ax_.set_axis_off() for ax_ in ax]
# Add legend
[ax_.legend(loc='upper left') for ax_ in ax];

In [ ]:
vizinhos_distancia.pct_nonzero

In [ ]:
#Alguns atributos da matrix
print("Número de regiões: ", vizinhos_distancia.n) 
print("Número de conexões:", vizinhos_distancia.nonzero)
print("Porcentagem de conexões:", vizinhos_distancia.pct_nonzero)
print("Município ilha: ", vizinhos_distancia.islands) # Municipios que são ilhas (sem conector)
print("Maior número de conexões: ", vizinhos_distancia.max_neighbors) 
print("Média número de conexões: ", vizinhos_distancia.mean_neighbors) 
print("Menor número de conexões: ", vizinhos_distancia.min_neighbors) 

# TO BE CONTINUED...

# Referências

* Referência: https://www.patriciaramos.org/espacial
* Pesos/Vizinhança: https://geographicdata.science/book/notebooks/04_spatial_weights.html
* https://splot.readthedocs.io/en/latest/installation.html
* https://pysal.org/libpysal/api.html